# Converting Tests into Yaml-Files 

In [ ]:
%matplotlib inline
# %load_ext lab_black

%load_ext autoreload
%autoreload 2


import pandas as pd
import yaml
from _gettsim_tests import TEST_DATA_DIR

In [ ]:
# reading in testfile in csv
file_name = "arbeitsl_geld.csv"
df = pd.read_csv(TEST_DATA_DIR / file_name, header=0, index_col=0, squeeze=True)

In [ ]:
df = df.reset_index()
df = df[df["jahr"] == 2019]
df

In [ ]:
out_cols = ["arbeitsl_geld_m"]
in_cols_assumed = [
    "anwartschaftszeit",
    "arbeitssuchend",
    "m_durchg_alg1_bezug",
    "soz_vers_pflicht_5j",
]
in_cols = [c for c in df if c not in out_cols]
in_cols_provided = [
    col for col in in_cols if col not in [*in_cols_assumed, "source", "note"]
]

In [ ]:
out = []
for hh_id in df["hh_id"].unique():
    df_hh = df.loc[df["hh_id"] == hh_id]
    source = df_hh["source"].iloc[0]
    note = df_hh["note"].iloc[0]
    specs = {"source": source, "note": note}
    inputs = {
        "provided": df_hh[in_cols_provided].to_dict("list"),
        "assumed": df_hh[in_cols_assumed].to_dict("list"),
    }
    outputs = {"outputs": df_hh[out_cols].to_dict("list")}
    out.append({"info": specs, "inputs": inputs, "outputs": outputs})
out

In [ ]:
text = yaml.dump(out, sort_keys=False, allow_unicode=True, width=89, indent=4)
print(text)

In [ ]:
with open("arbeitsl_geld.yaml", "w") as text_file:
    text_file.write(text)